In [43]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
validation = pd.read_csv('./validation.csv')
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')

In [45]:
train_text = train['text']
train_labels = train['label']

In [46]:
train_text[0], train_labels[0]

('i didnt feel humiliated', 0)

In [47]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>', split=' ', char_level=False)
tokenizer.fit_on_texts(train_text)

In [48]:
train_text[0]

'i didnt feel humiliated'

In [49]:
tokenizer.texts_to_sequences([train_text[0]])

[[2, 139, 3, 679]]

In [50]:
maxlen=50
def get_sequences(tokenizer, train_text):
    sequences = tokenizer.texts_to_sequences(train_text)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

In [51]:
padded_train_seq = get_sequences(tokenizer, train_text)

In [52]:
padded_train_seq[0]

array([  2, 139,   3, 679,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [53]:
classes = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

In [54]:
print(classes)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [55]:
class_to_index

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [56]:
index_to_class

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [57]:
print(train_labels[0])

0


In [58]:
'''model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])'''
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,64,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(80)),
tf.keras.layers.Dense(6, activation='sigmoid')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
     metrics=['accuracy']
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 64)            640000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 160)               92800     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 6)                 966       
                                                                 
Total params: 733766 (2.80 MB)
Trainable params: 733766 (2.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
val_text = validation['text']
val_labels = validation['label']
val_seq = get_sequences(tokenizer, val_text)

In [60]:
h = model.fit(
     padded_train_seq, train_labels, batch_size = 20,
     validation_data=(val_seq, val_labels),
     epochs=30,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.01, patience=5)]
)

Epoch 1/30
800/800 [==============================] - 10s 12ms/step - loss: 1.2575 - accuracy: 0.5166 - val_loss: 0.6823 - val_accuracy: 0.7545
Epoch 2/30
800/800 [==============================] - 9s 12ms/step - loss: 0.4290 - accuracy: 0.8587 - val_loss: 0.3919 - val_accuracy: 0.8590
Epoch 3/30
800/800 [==============================] - 9s 11ms/step - loss: 0.2055 - accuracy: 0.9354 - val_loss: 0.3852 - val_accuracy: 0.8755
Epoch 4/30
800/800 [==============================] - 9s 11ms/step - loss: 0.1333 - accuracy: 0.9571 - val_loss: 0.4021 - val_accuracy: 0.8890
Epoch 5/30
800/800 [==============================] - 9s 11ms/step - loss: 0.1024 - accuracy: 0.9661 - val_loss: 0.3705 - val_accuracy: 0.8970
Epoch 6/30
800/800 [==============================] - 9s 11ms/step - loss: 0.0863 - accuracy: 0.9711 - val_loss: 0.3010 - val_accuracy: 0.9055
Epoch 7/30
800/800 [==============================] - 9s 11ms/step - loss: 0.0634 - accuracy: 0.9785 - val_loss: 0.3272 - val_accuracy: 0.906

In [61]:
test_text = test['text']
test_labels=test['label']
test_seq = get_sequences(tokenizer, test_text)

model.evaluate(test_seq, test_labels)

63/63 [==============================] - 1s 5ms/step - loss: 0.3979 - accuracy: 0.8845


[0.3978656530380249, 0.8845000267028809]

In [62]:
model.save("./emotion_model.h5")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
